# Importing Data
- Scraping names and committee names of Senate Committee Members
- Scraping individual data from wikipedia from the previous list created
- Creating dataframes of both committee memebers ranking member and chair 
- including voter data, state, assumed office, time in office

In [38]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime

In [39]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_Senate_committees"

In [40]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')
#print(soup.prettify())
names = pd.read_html(url)

In [41]:
#columnNames = smallDF.columns().tolist()
sliceNames = names[0]

In [42]:
# names[0].head()

In [43]:
sliceNames.to_csv('names.csv', index = False)

In [44]:
importNames = pd.read_csv('names.csv')
committee = importNames['Committee']
chairNames = importNames['Chair']
rankingMembers = importNames['Ranking Member']

In [45]:
chairNames = list(chairNames)
chairNames = chairNames[1:]
rankingMembers = rankingMembers[1:]

In [46]:
#Pulling out list of names from scrape
#chairNames[1][:12]
def stripNames(stripList):
    stateParty = []
    fullName = []
    party = []
    state = []
    for i in stripList:
        counter = 0
        for j in i:
            counter += 1
            if j == '(':
                stateParty.append(i[counter-1:])
                fullName.append(i[:counter-2])
                party.append(i[-5:len(i)-4])
                state.append(i[-3:len(i)-1])
                #print(counter)
    return fullName, stateParty, party, state

In [47]:
importNames

,Committee,Committee.1,Chair,Ranking Member,Refs
0,Unnamed: 0_level_1,Subcommittee,Chair,Ranking Member,Refs
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts (R-KS),Debbie Stabenow (D-MI),[1][2]
2,NaN,"Commodities, Risk Management and Trade",John Boozman (R-AR),Sherrod Brown (D-OH),[3]
3,NaN,"Conservation, Forestry and Natural Resources",Mike Braun (R-IN),Michael Bennet (D-CO),[3]
4,NaN,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith (R-MS),Kirsten Gillibrand (D-NY),[3]
...,...,...,...,...,...
80,NaN,Intellectual Property,Thom Tillis (R-NC),Chris Coons (D-DE),[14]
81,NaN,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse (R-NE),Richard Blumenthal (D-CT),[14]
82,Rules and Administration,Rules and Administration,Roy Blunt (R-MO),Amy Klobuchar (D-MN),[1][2]
83,Small Business and Entrepreneurship,Small Business and Entrepreneurship,Marco Rubio (R-FL),Ben Cardin (D-MD),[1][2]


In [48]:
chairNamesCols = stripNames(chairNames)
rankingMembersCols = stripNames(rankingMembers)
len(importNames)
importNames = importNames.drop(importNames.index[0])
print(len(importNames))

84


In [49]:
importNames['Chair Names'] = chairNamesCols[0]
importNames['Party'] = chairNamesCols[1]
importNames['PartyAbbv'] = chairNamesCols[2]
importNames['State'] = chairNamesCols[3]
importNames['Ranking Members'] = rankingMembersCols[0]
importNames['Party'] = rankingMembersCols[1]
importNames['PartyAbbv'] = rankingMembersCols[2]
importNames['State'] = rankingMembersCols[3]

In [53]:
importNames = importNames.rename(columns={'Committee.1':'SubComittee'})
importNames['Committee'] = importNames['Committee'].replace(np.NaN, 'SubCommittee')
importNames.drop(columns=['Refs', 'Ranking Member', 'Chair', 'Party'])

,Committee,SubComittee,Chair Names,PartyAbbv,State,Ranking Members
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts,D,MI,Debbie Stabenow
2,SubCommittee,"Commodities, Risk Management and Trade",John Boozman,D,OH,Sherrod Brown
3,SubCommittee,"Conservation, Forestry and Natural Resources",Mike Braun,D,CO,Michael Bennet
4,SubCommittee,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith,D,NY,Kirsten Gillibrand
5,SubCommittee,"Nutrition, Agricultural Research and Specialty...",Deb Fischer,D,PA,Bob Casey
...,...,...,...,...,...,...
80,SubCommittee,Intellectual Property,Thom Tillis,D,DE,Chris Coons
81,SubCommittee,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse,D,CT,Richard Blumenthal
82,Rules and Administration,Rules and Administration,Roy Blunt,D,MN,Amy Klobuchar
83,Small Business and Entrepreneurship,Small Business and Entrepreneurship,Marco Rubio,D,MD,Ben Cardin


In [ ]:
# This should work???
# def replaceUnderscore(replaceList):
#     names_= []
#     i=0
#     while i < len(replaceList):
#         names_.append((replaceList[0][i]).replace(' ', '_'))
#         i += 1 
#     return names_
# print(replaceUnderscore(chairNamesCols))

In [ ]:
chairNames_= []
rankingNames_=[]
i=0
while i < len(chairNames):
    rankingNames_.append((chairNamesCols[0][i]).replace(' ', '_'))
    chairNames_.append((chairNamesCols[0][i]).replace(' ', '_'))
    i += 1

In [ ]:
count = chairNames_
j = 0
k = 0
for l in chairNames_:
    k = 0
    while k < 14:
        count = allRecords[j]
        globals()[l].append(count)
        k += 1
    j += 1

In [ ]:
count = chairNames_
for l in chairNames_:
    globals()l = []
    while k < 14:
        count = allRecords[j]
        globals()[l].append(count)
        k += 1
    j += 1

In [ ]:
allRecords = []
counter = 1
for i in chairNames_:
        totalRequests = len(chairNames_)
        print('////////////////////////////////////////////////////////////////////')
        print(f'REQUESTING {totalRequests*14} RECORDS')
        time.sleep(10)
        if i == 'VACANT':
            print('skipping')
            counter += 1
            continue
        else:
            try:
                print('////////////////////////////////////////////////////////////////////')
                print(f'RETRIEVING RECORDS:: {i}, {counter} OF {totalRequests}')
                print('////////////////////////////////////////////////////////////////////')
                SenatorUrl = 'https://en.wikipedia.org/wiki/' + i
                counter14 = 0
                while counter14 < 14:
                    holdingVar = (pd.read_html(SenatorUrl)[counter14])
                    print('////////////////////////////////////////////////////////////////////')
                    print(f'TOTAL RECORDS RECIEVED {len(holdingVar)}: REQUESTING {counter14} OF 14 FOR {i}')
                    print('////////////////////////////////////////////////////////////////////')
                    print(holdingVar)
                    allRecords.append(holdingVar)
                    counter14 += 1 
                    counter += 1
            except:
                print('VALUE ERROR OR INDEX ERROR')
                print('////////////////////////////////////////////////////////////////////')
                continue
    

In [ ]:
len(allRecords)

In [ ]:
type(allRecords[0])

In [ ]:
chairNames_[3]

In [ ]:
count = chairNames_
j = 0
k = 0
for l in chairNames_:
    globals()l = []
    while k < 14:
        count = allRecords[j]
        globals()[l].append(count)
        k += 1
    j += 1

In [ ]:
Mike_Braun

In [ ]:
len(John_Boozman)

In [ ]:
Pat_Roberts

In [ ]:
#ConvertToDict(allRecords)

In [ ]:
allRecords.to_csv('wikiRecords', index = False)